In [2]:
import pandas as pd
from utils import *
import os

In [3]:
df = pd.read_csv(f"../data/dataset.csv")
df.head()

,timestamp,id,bikes_available,lat,lon,hour,temperature_celsius,precipitation_mm,humidity_percent,wind_speed_kmh,...,is_weekend,month_sin,month_cos,lag_1,lag_4,lag_8,lag_12,lag_24,roll_mean_4,roll_mean_8
0,2024-12-01 06:15:00,1,1,39.480042,-0.382929,6,11.575,0.0,83.00,5.3,...,1,-2.449294e-16,1.0,1.0,1.0,0.0,2.0,1.0,1.0,0.750
1,2024-12-01 06:30:00,1,1,39.480042,-0.382929,6,11.550,0.0,83.00,5.2,...,1,-2.449294e-16,1.0,1.0,1.0,0.0,2.0,3.0,1.0,0.875
2,2024-12-01 06:45:00,1,1,39.480042,-0.382929,6,11.525,0.0,83.00,5.1,...,1,-2.449294e-16,1.0,1.0,1.0,1.0,0.0,3.0,1.0,1.000
3,2024-12-01 07:00:00,1,1,39.480042,-0.382929,7,11.500,0.0,83.00,5.0,...,1,-2.449294e-16,1.0,1.0,1.0,1.0,0.0,2.0,1.0,1.000
4,2024-12-01 07:15:00,1,0,39.480042,-0.382929,7,11.475,0.0,83.25,5.0,...,1,-2.449294e-16,1.0,1.0,1.0,1.0,0.0,2.0,1.0,1.000


In [4]:
distances = pd.read_csv(f"../data/distance_matrix.csv")
distances.head()

,id,1,10,100,101,102,103,104,106,107,...,90,91,92,93,94,95,96,97,98,99
0,1,0.00,1205.50,4676.66,4982.70,4016.19,3672.19,4170.48,4540.13,5196.34,...,2783.26,4836.61,2969.10,4128.37,3525.53,4303.88,3739.28,4346.31,4011.17,4488.00
1,10,1205.50,0.00,5151.42,5486.41,4490.95,4281.67,4680.40,5050.05,5671.11,...,3065.52,3847.16,3472.81,4632.09,4030.52,4913.36,4243.00,4821.07,3914.37,4991.72
2,100,4676.66,5151.42,0.00,474.85,1040.39,1710.18,1440.57,908.98,532.40,...,1991.78,4915.71,1729.74,665.12,1283.65,1574.82,987.07,462.47,1490.31,277.22
3,101,5256.45,5731.21,592.51,0.00,1620.19,2289.97,1793.81,1262.23,831.69,...,2571.57,5039.05,2309.53,1238.93,1863.44,2065.63,1560.89,1042.26,2064.13,844.83
4,102,4003.14,4486.89,1040.39,1502.24,0.00,1036.66,399.51,671.09,1259.72,...,1318.26,4762.75,1065.21,648.32,597.03,711.95,375.77,710.04,905.43,1007.55


In [5]:
times = pd.read_csv(f"../data/duration_matrix.csv")
times.head()

,id,1,10,100,101,102,103,104,106,107,...,90,91,92,93,94,95,96,97,98,99
0,1,0.00,259.05,971.01,1036.45,855.38,776.18,891.55,954.95,1079.03,...,590.89,999.44,635.07,885.05,757.56,914.30,787.79,928.83,820.36,950.07
1,10,259.05,0.00,1050.18,1114.63,934.54,875.50,977.74,1041.14,1158.19,...,639.35,803.13,713.25,963.22,842.77,1013.63,865.96,1007.99,793.47,1028.25
2,100,971.01,1050.18,0.00,97.50,229.60,370.13,292.20,186.37,115.64,...,432.00,997.37,363.33,155.00,269.58,336.08,199.94,121.45,300.58,79.37
3,101,1092.44,1171.60,129.06,0.00,351.03,491.56,368.86,263.03,180.97,...,553.42,1095.82,484.76,275.24,391.00,442.25,320.18,242.88,420.81,189.52
4,102,836.69,917.65,229.60,319.42,0.00,235.81,106.87,162.65,262.90,...,297.67,977.51,230.80,176.84,132.63,186.38,94.14,187.43,200.06,233.04


Preparamos las matrices

In [6]:
# df_distancias o df_tiempos ya cargado
# Asegúrate de que la primera columna se llama "id"
distances.set_index('id', inplace=True)
times.set_index('id', inplace=True)

# Conversión a diccionario de diccionarios
D = distances.to_dict(orient='index') #diccionario de distancias
# Convertimos todas las claves internas a int
D = {
    ext_k: {int(inner_k): v for inner_k, v in inner_dict.items()}
    for ext_k, inner_dict in D.items()
}


T = times.to_dict(orient='index') #diccionario de tiempos
T = {
    ext_k: {int(inner_k): v for inner_k, v in inner_dict.items()}
    for ext_k, inner_dict in T.items()
}
D


{1: {1: 0.0,
  10: 1205.5,
  100: 4676.66,
  101: 4982.7,
  102: 4016.19,
  103: 3672.19,
  104: 4170.48,
  106: 4540.13,
  107: 5196.34,
  108: 5363.3,
  109: 5387.23,
  11: 2113.16,
  110: 5373.77,
  111: 5041.85,
  112: 4672.19,
  113: 4556.98,
  114: 4320.27,
  115: 3623.14,
  116: 3089.83,
  117: 3048.91,
  118: 2626.68,
  119: 2602.87,
  12: 2378.42,
  120: 2537.2,
  121: 2921.63,
  122: 3711.35,
  123: 2549.98,
  124: 2850.84,
  125: 3162.02,
  126: 3055.17,
  127: 2589.83,
  128: 2349.58,
  129: 1838.51,
  13: 1991.56,
  130: 1605.73,
  131: 1197.87,
  132: 1409.05,
  133: 927.82,
  134: 1642.93,
  135: 1022.93,
  136: 1574.03,
  137: 1489.45,
  138: 1118.86,
  139: 825.76,
  14: 1597.09,
  140: 1165.31,
  141: 452.24,
  142: 894.36,
  143: 994.58,
  144: 1823.22,
  145: 1897.14,
  147: 1392.97,
  148: 2018.69,
  149: 3297.85,
  15: 2023.77,
  150: 3695.23,
  151: 4915.2,
  152: 3049.54,
  153: 590.74,
  154: 5949.69,
  155: 2966.39,
  156: 2907.28,
  157: 2009.69,
  158: 5973.

In [7]:
df_bikes = pd.read_csv(f"../data/bike_stations.csv")
df_bikes.head()

,id,address,total_spaces,geo_shape,lat,lon
0,1,C/GUILLEM DE CASTRO esquina con C/NA JORDANA,25,"{""coordinates"": [-0.38292927973315827, 39.4800...",39.480042,-0.382929
1,16,"Colón, 60",20,"{""coordinates"": [-0.3704333693720587, 39.47009...",39.470092,-0.370433
2,137,Economista Gay - Luis Crumiere,20,"{""coordinates"": [-0.3798963873712934, 39.48903...",39.489033,-0.379896
3,148,C/Xàtiva 30-32,35,"{""coordinates"": [-0.3753782937388751, 39.46714...",39.467145,-0.375378
4,150,Manuel Candela - Rodriguez de Cepeda,25,"{""coordinates"": [-0.35060930364544307, 39.4677...",39.467721,-0.350609


In [8]:
df.columns

Index(['timestamp', 'id', 'bikes_available', 'lat', 'lon', 'hour',
       'temperature_celsius', 'precipitation_mm', 'humidity_percent',
       'wind_speed_kmh', 'weekday', 'is_weekend', 'month_sin', 'month_cos',
       'lag_1', 'lag_4', 'lag_8', 'lag_12', 'lag_24', 'roll_mean_4',
       'roll_mean_8'],
      dtype='object')

Realizamos las predicciones


In [9]:
if not pd.api.types.is_datetime64_any_dtype(df['timestamp']):
        df = df.copy()
        df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

is_may_2025 = (
        (df['timestamp'].dt.year  == 2025) &
        (df['timestamp'].dt.month == 5)
)
mayo = df.loc[is_may_2025].copy()

# Load the retrained model
# model_path = os.path.join('model', 'retrained_model.pkl')
model = joblib.load(f"../model/retrained_model.pkl")

# Prepare features and run inference
TARGET = 'bikes_available'
feature_cols = [c for c in mayo.columns if c not in ('timestamp', TARGET)]
X_test = mayo[feature_cols]
mayo['predicted'] = model.predict(X_test)

Calculamos cuantos huecos hay libres en cada momento y estación.

In [10]:
mayo_filtered = mayo[['timestamp','id', 'predicted']]
mayo_merged = mayo_filtered.merge(df_bikes, on='id', how='left')

# Calculamos los espacios disponibles previstos
mayo_merged['available_spaces'] = mayo_merged['total_spaces'] - round(mayo_merged['predicted'])
mayo_merged.head()
    

,timestamp,id,predicted,address,total_spaces,geo_shape,lat,lon,available_spaces
0,2025-05-01 00:00:00,1,2.009686,C/GUILLEM DE CASTRO esquina con C/NA JORDANA,25,"{""coordinates"": [-0.38292927973315827, 39.4800...",39.480042,-0.382929,23.0
1,2025-05-01 00:15:00,1,2.980255,C/GUILLEM DE CASTRO esquina con C/NA JORDANA,25,"{""coordinates"": [-0.38292927973315827, 39.4800...",39.480042,-0.382929,22.0
2,2025-05-01 00:30:00,1,2.975880,C/GUILLEM DE CASTRO esquina con C/NA JORDANA,25,"{""coordinates"": [-0.38292927973315827, 39.4800...",39.480042,-0.382929,22.0
3,2025-05-01 00:45:00,1,2.013927,C/GUILLEM DE CASTRO esquina con C/NA JORDANA,25,"{""coordinates"": [-0.38292927973315827, 39.4800...",39.480042,-0.382929,23.0
4,2025-05-01 01:00:00,1,1.077575,C/GUILLEM DE CASTRO esquina con C/NA JORDANA,25,"{""coordinates"": [-0.38292927973315827, 39.4800...",39.480042,-0.382929,24.0


In [11]:
df_preds = mayo_merged.set_index(["id", "timestamp"])
P = df_preds["available_spaces"].to_dict()
P

{(1, Timestamp('2025-05-01 00:00:00')): 23.0,
 (1, Timestamp('2025-05-01 00:15:00')): 22.0,
 (1, Timestamp('2025-05-01 00:30:00')): 22.0,
 (1, Timestamp('2025-05-01 00:45:00')): 23.0,
 (1, Timestamp('2025-05-01 01:00:00')): 24.0,
 (1, Timestamp('2025-05-01 01:15:00')): 23.0,
 (1, Timestamp('2025-05-01 01:30:00')): 23.0,
 (1, Timestamp('2025-05-01 01:45:00')): 23.0,
 (1, Timestamp('2025-05-01 02:00:00')): 23.0,
 (1, Timestamp('2025-05-01 02:15:00')): 23.0,
 (1, Timestamp('2025-05-01 02:30:00')): 23.0,
 (1, Timestamp('2025-05-01 02:45:00')): 23.0,
 (1, Timestamp('2025-05-01 03:00:00')): 24.0,
 (1, Timestamp('2025-05-01 03:15:00')): 24.0,
 (1, Timestamp('2025-05-01 03:30:00')): 24.0,
 (1, Timestamp('2025-05-01 03:45:00')): 24.0,
 (1, Timestamp('2025-05-01 04:00:00')): 24.0,
 (1, Timestamp('2025-05-01 04:15:00')): 24.0,
 (1, Timestamp('2025-05-01 04:30:00')): 24.0,
 (1, Timestamp('2025-05-01 04:45:00')): 25.0,
 (1, Timestamp('2025-05-01 05:00:00')): 25.0,
 (1, Timestamp('2025-05-01 05:15:0

In [28]:
from heapq import heappush, heappop
from datetime import datetime, timedelta

def round_to_next_15min(dt):
    # Si ya es múltiplo de 15, no redondeamos
    if dt.minute % 15 == 0 and dt.second == 0 and dt.microsecond == 0:
        return dt.replace(second=0, microsecond=0)
    
    # Si no lo es, redondeamos hacia el siguiente múltiplo de 15
    minute = ((dt.minute // 15) + 1) * 15
    if minute == 60:
        dt += timedelta(hours=1)
        minute = 0
    return dt.replace(minute=minute, second=0, microsecond=0)


def a_star_con_distancia(start, goal, salida_datetime, T, D, P, max_tiempo=30):
    open_set = []
    heappush(open_set, (0, start, salida_datetime, [start]))
    visited = set()

    while open_set:
        #sacamos el nodo con manor f (sería como la cota que calculamos)
        f, actual, tiempo_llegada, camino = heappop(open_set)
        #print(f"Visiting: {actual}, Arrival time: {tiempo_llegada}, Path: {camino}")  # Debugging line to check current node and path
        # redondeamos el tiempo de llegada al siguiente múltiplo de 15 minutos
        tiempo_pred = round_to_next_15min(tiempo_llegada)
        #print(f"Visiting: {actual} at {tiempo_pred}, path: {camino}")  # Debugging line to check current node and path
        #print(f"Predicted time: {tiempo_pred}, Arrival time: {tiempo_llegada}")  # Debugging line to check times
        #print(f"Available spaces at {actual} at {tiempo_pred}: {P.get((actual, tiempo_pred), 0)}")  # Debugging line to check available spaces
        # comprobamos si al menos hay un hueco disponible de anclaje
        start = int(start)
        goal = int(goal)
        if actual == goal:
            print ('Llegamos al destino:', actual)
    # Verificamos huecos en la estación de destino
            if P.get((goal, tiempo_pred), 0) >= 1:
                return camino  # ✅ podemos anclar, fin
            else:
                # ❌ No hay huecos en goal → buscamos otra estación alternativa cercana
                alternativas = []
                for alterna in T.get(goal, {}):
                    duracion_extra = T[goal][alterna] / 60  # asumimos segundos → minutos
                    if duracion_extra > max_tiempo:
                        continue

                    llegada_alterna = tiempo_llegada + timedelta(minutes=duracion_extra)
                    llegada_alterna_pred = round_to_next_15min(llegada_alterna)

                    if P.get((alterna, llegada_alterna_pred), 0) >= 1:
                        alternativas.append((duracion_extra, alterna, llegada_alterna, llegada_alterna_pred))

                if alternativas:
                    # Escogemos la estación más cercana (menos tiempo adicional)
                    alternativas.sort()
                    mejor = alternativas[0]
                    _, mejor_id, llegada, llegada_pred = mejor

                    # Recalculamos heurística y coste acumulado
                    h = D.get(mejor_id, {}).get(goal, float('inf'))
                    g = (llegada - salida_datetime).total_seconds() / 60
                    f_score = g + h

                    print(f"⚠️ Estación destino sin huecos. Usamos alternativa: {mejor_id} (llegada {llegada_pred})")
                    heappush(open_set, (f_score, mejor_id, llegada, camino + [mejor_id]))
                    continue  # no terminamos todavía, dejamos que el algoritmo continúe
                else:
                    print(f"❌ No hay huecos ni en el destino ni en estaciones cercanas accesibles en menos de {max_tiempo} minutos.")
                    return None
        # Si no es el destino, añadimos la estación actual al conjunto de visitados
        if actual != start and P.get((actual, tiempo_pred), 0) < 1 and actual != goal:
            continue
        print(f"\n🚴 Visitando: {actual}")
        print(f"  ⏱ Llegada real: {tiempo_llegada} → redondeado a: {tiempo_pred}")
        print(f"  📦 Huecos disponibles: {P.get((actual, tiempo_pred), 0)}")
        print(f"  🧭 Ruta parcial: {camino}")

        visited.add((actual, tiempo_pred)) #añadimos la estacion que acabamos de visitar y el tiempo predicho de llegada
        #print(f"Open set size: {len(open_set)}")  # Debugging line to check open_set size
        #print(f"Visited size: {len(visited)}")  # Debugging line to check visited size
        
        for vecino in T.get(actual, {}): #buscamos los vecinos de la estacion actual
             # Debugging line to check neighbors
            duracion = T[actual][vecino] #nos quedamos con el tiempo desde la estacion actual al vecino
            duracion = round(duracion / 60)  # Convertimos a minutos
            #print(f"Checking neighbor: {vecino}, duration: {duracion} minutes")  # Debugging line to check neighbor and duration
            if duracion > max_tiempo: #si el tiempo de duración es mayor que el máximo permitido, lo saltamos
                continue # saltamos el resto del bucle

            nuevo_tiempo = tiempo_llegada + timedelta(minutes=duracion) # calculamos el nuevo tiempo de llegada para el vecino, usamos timedelta porque la duration matrix está en segundos
            nuevo_tiempo_pred = round_to_next_15min(nuevo_tiempo) #redondeamos de nuevo a lbloque de 15 min
            
            if (vecino, nuevo_tiempo_pred) in visited: #comporbamos que no hemos visitado ya el vecino en ese tiempo predicho
                continue

            if P.get((vecino, nuevo_tiempo_pred), 0) < 1: # si hay o no hueco disponible en el vecino
                continue

            # Heurística: distancia desde vecino al destino, que sería la cota optimista que usamos 
            h = D.get(vecino, {}).get(goal, float('inf'))  # km entre vecino y destino
            g = (nuevo_tiempo - salida_datetime).total_seconds() / 60  # tiempo acumulado en minutos
            f_score = g + h  # puedes ponderar h si quieres convertir a tiempo estimado

        


            heappush(open_set, (f_score, vecino, nuevo_tiempo, camino + [vecino]))
        

    return None  # no hay ruta válida


In [13]:
ahora = datetime(2025, 6, 19, 11, 13)  # 11:13 AM
#redondear al próximo bloque de 15 minutos
round_to_next_15min(ahora)

datetime.datetime(2025, 6, 19, 11, 15)

In [29]:
salida_datetime = datetime(2025, 5, 1, 11, 0) 
ruta = a_star_con_distancia(268, 112, salida_datetime, T, D, P, max_tiempo=30)
print("Ruta encontrada:", ruta)
#preguntar a andreu que le parece



🚴 Visitando: 268
  ⏱ Llegada real: 2025-05-01 11:00:00 → redondeado a: 2025-05-01 11:00:00
  📦 Huecos disponibles: 5.0
  🧭 Ruta parcial: [268]

🚴 Visitando: 95
  ⏱ Llegada real: 2025-05-01 11:30:00 → redondeado a: 2025-05-01 11:30:00
  📦 Huecos disponibles: 26.0
  🧭 Ruta parcial: [268, 95]
Llegamos al destino: 112
Ruta encontrada: [268, 95, 112]


In [ ]:
salida_datetime = datetime(2025, 5, 1, 15,30) 
ruta = a_star_con_distancia(268, 168, salida_datetime, T, D, P, max_tiempo=30)
print("Ruta encontrada:", ruta)


🚴 Visitando: 268
  ⏱ Llegada real: 2025-05-01 15:30:00 → redondeado a: 2025-05-01 15:30:00
  📦 Huecos disponibles: 4.0
  🧭 Ruta parcial: [268]

🚴 Visitando: 1
  ⏱ Llegada real: 2025-05-01 15:48:00 → redondeado a: 2025-05-01 16:00:00
  📦 Huecos disponibles: 24.0
  🧭 Ruta parcial: [268, 1]

🚴 Visitando: 2
  ⏱ Llegada real: 2025-05-01 15:50:00 → redondeado a: 2025-05-01 16:00:00
  📦 Huecos disponibles: 14.0
  🧭 Ruta parcial: [268, 2]

🚴 Visitando: 2
  ⏱ Llegada real: 2025-05-01 15:51:00 → redondeado a: 2025-05-01 16:00:00
  📦 Huecos disponibles: 14.0
  🧭 Ruta parcial: [268, 1, 2]

🚴 Visitando: 3
  ⏱ Llegada real: 2025-05-01 15:51:00 → redondeado a: 2025-05-01 16:00:00
  📦 Huecos disponibles: 17.0
  🧭 Ruta parcial: [268, 3]

🚴 Visitando: 3
  ⏱ Llegada real: 2025-05-01 15:52:00 → redondeado a: 2025-05-01 16:00:00
  📦 Huecos disponibles: 17.0
  🧭 Ruta parcial: [268, 1, 3]

🚴 Visitando: 3
  ⏱ Llegada real: 2025-05-01 15:53:00 → redondeado a: 2025-05-01 16:00:00
  📦 Huecos disponibles: 17.0
 

In [16]:
tiempo_pred = datetime(2025, 5, 1, 15,30) 
P.get((168, tiempo_pred), 0)

0